<a href="https://colab.research.google.com/github/Bateyjosue/NLP_Fellowship/blob/main/AssignmentNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Model Assignment
The aim of this assignment is to make sure you understand the foundations of model training. We have covered traditional ML and also simple NN. You task is 
1. to write code for training traditional ML model that gives the highest accuracy
2. Code for NN model that gives the highest accuracy

## This we will consider
1. The code works
2. The hyperparameter used for fine tuning - epoch only is not enough
3. The highest accuracy
4. Bonus points for being creative with preprocessing, tokenization and creation of vectors

# Submissions
1. Notebook with code
2. 2 models

# Deadline
Monday 28th at 5pm

In [20]:
import pandas as pd

dataset = pd.read_csv('https://raw.githubusercontent.com/agent87/NLP-FELLOWSHIP/main/Data/50k_imdb_movie_reviews.csv')

dataset.head(10)

,review,sentiment,set
0,I went and saw this movie last night after bei...,1,test
1,Actor turned director Bill Paxton follows up h...,1,test
2,As a recreational golfer with some knowledge o...,1,test
3,"I saw this film in a sneak preview, and it is ...",1,test
4,Bill Paxton has taken the true story of the 19...,1,test
5,"I saw this film on September 1st, 2005 in Indi...",1,test
6,"Maybe I'm reading into this too much, but I wo...",1,test
7,I felt this film did have many good qualities....,1,test
8,This movie is amazing because the fact that th...,1,test
9,"""Quitting"" may be as much about exiting a pre-...",1,test


In [24]:
dataset[(dataset['sentiment'] == 1) & (dataset['set'] == 'train')]

,review,sentiment,set
25000,Bromwell High is a cartoon comedy. It ran at t...,1,train
25001,Homelessness (or Houselessness as George Carli...,1,train
25002,Brilliant over-acting by Lesley Ann Warren. Be...,1,train
25003,This is easily the most underrated film inn th...,1,train
25004,This is not the typical Mel Brooks film. It wa...,1,train
...,...,...,...
37495,"Seeing as the vote average was pretty low, and...",1,train
37496,"The plot had some wretched, unbelievable twist...",1,train
37497,I am amazed at how this movie(and most others ...,1,train
37498,A Christmas Together actually came before my t...,1,train


In [ ]:
print("No of positive in train : "+str(len(dataset[(dataset['sentiment'] == 1) & (dataset['set'] == 'train')])))
print("No of positive in train : "+str(len(dataset[(dataset['sentiment'] == 1) & (dataset['set'] == 'test')])))


In [28]:
dataset = dataset.drop_duplicates()

In [ ]:
train_dataset = dataset[(dataset['set'] == 'train')]
test_dataset = dataset[(dataset['set'] == 'test')]

print(len(train_dataset))
print(len(test_dataset))

In [38]:
import re
def preprocessing(texts):
  cleaned_text = [] #Construct an empty list to store the reviews
  for text in texts: #for review in df['reviews']
    text = text.lower() #lower case the text
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    
    url_pattern = re.compile(r'https?://\S+|www\.\S+') #compile the urls/links patterns
    html_pattern = re.compile('<.*?>') #compile the html tafs

    text = emoji_pattern.sub(r'', text) #replace emojis
    text = url_pattern.sub(r'', text) #replace
    text = html_pattern.sub(r'', text)
    text = re.sub(r"[^\w\d'\s]+", ' ', text) # remove special characters!
    
    cleaned_text.append(text)

  return cleaned_text 

In [41]:
len(dataset.iloc[0].review.split(' '))

152

In [43]:
df = dataset

In [45]:
size = []
for index, review in enumerate(df.review):
  size.append(len(review.split(' ')))

df['length'] = size

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [46]:
df.loc[2470]

review       I like the show, but come-on writers, get some...
sentiment                                                    1
set                                                       test
length                                                     154
Name: 2470, dtype: object

In [ ]:
df.describe()

In [ ]:
!pip install torch==1.8.0 torchtext==0.9.0

In [51]:
import torch #used to build the model/neural network

from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator

SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
max_document_length = 300
TEXT = data.Field(lower=True, include_lengths=True, tokenize='spacy', preprocessing=preprocessing, batch_first=True, fix_length=max_document_length)
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
  def __init__(self, df: pd.DataFrame, fields: list):
    # items = []
    # for i, r in df.iterrows():
    #   obj = Example.fromlist(list(r), fields)
    #   items.append(obj)
    # print(items)
    # [Example.fromlist(list(r), fields) for i, r in df.iterrows()]

    super(DataFrameDataset, self).__init__(
      [Example.fromlist(list(r), fields) for i, r in df.iterrows()], 
      fields
    )

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [57]:
torch_valid_dataset, torch_test_dataset = DataFrameDataset(
  df=test_dataset, fields=(
      ('review', TEXT),
      ('sentiment', LABEL)
  )
).split()

In [59]:
torch_valid_dataset

In [62]:
torch_train_dataset = DataFrameDataset(
  df=train_dataset,
  fields=(
    ('review', TEXT),
    ('sentiment', LABEL)
  )
)

In [63]:
max_size = 30000
TEXT.build_vocab(torch_train_dataset, max_size=max_size, vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)

vocab_size

100%|██████████| 111051/111051 [00:11<00:00, 9707.51it/s] 


30002

In [64]:
print(TEXT.vocab.freqs.most_common(20))

[(' ', 789819), ('the', 327886), ('and', 162468), ('a', 161303), ('of', 145163), ('to', 134991), ('is', 109842), ('it', 93209), ('in', 92566), ('i', 82429), ('this', 73164), ('that', 72862), ("'s", 61935), (' br', 52529), ('was', 50232), ('as', 46027), ('for', 43809), ('with', 43716), ('movie', 42925), ('but', 41504)]


In [65]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
  (torch_train_dataset, torch_valid_dataset, torch_test_dataset),
  batch_size=BATCH_SIZE,
  sort_key=lambda x: len(x.review),
  sort_within_batch=True
)